In [1]:
# LLMChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain



# Extracción de las entidades nombradas

## Prompt

In [94]:
TEMPLATE_NER = """
Como periodista y científico de redes, está analizando la red social que surgió de un Escándalo de corrupción en Colombia.
Su tarea es inferir y reconocer las entidades nombradas de organizaciones e individuos de la noticia entre ```.
Algunas de las entidades no son explicitas, por lo que debe inferirlas a partir de la información de la noticia.


La salida solo deberia ser en formato JSON con las siguientes claves:

"Organizaciones": Lista de Nombres de Organizaciones 
"Individuos": Lista de diccionarios con los nombres de los individuos y su rol en el escándalo:
    "Nombre"
    "Cargo"
    "rolEnEscandalo"
"Relaciones": Lista de triplas de las relaciones entre individuos y organizaciones, con la siguiente estructura:
    "Individuo"
    "Relación"
    "Organización"
    
Los clasificadores de relaciones deben tener una frase verbal como ejemplo (nacio) + frase preposicional (EnCiudad) -> nacioenCiudad:\n 

```
{article}
```
"""

PROMPT_NER = PromptTemplate(input_variables=["article"], template=TEMPLATE_NER)

## Parsear la Salida

In [ ]:
# Output Parser
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

class Person(BaseModel):
    name: str
    position: str = None
    
    

## Definición de la Cadena

In [29]:
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.2, max_tokens=8000)

llm_chain_NER = LLMChain(
    llm=llm,
    prompt=PROMPT_NER,
    #output_parse_function=parse_output,
)

## Ejemplo

In [38]:
input = {"article":"""
Luego de una extensa investigación, la Contraloría General de la República profirió un fallo de responsabilidad fiscal por más de 2,9 billones de pesos por el daño producido a los intereses del Estado debido, según se lee en el documento, a las decisiones sobre el manejo de los recursos públicos que "conllevaron a la pérdida de valor de las mayores inversiones en el Proyecto de Ampliación y Modernización de la Refinería de Cartagena ( Reficar )". La decisión se conoció en un auto firmado este 26 de abril por la Contralora Delegada Intersectorial n.° 15 de la Unidad de Investigaciones Especiales contra la Corrupción. (Puede ser de su interés: Recursos por $ 6,6 billones están en juego en obras inconclusas) Es así como, por el mayor daño patrimonial al Estado en su historia, el órgano de control llamó a responder fiscalmente a dos presidentes y tres vicepresidentes de Reficar, siete miembros de su junta directiva (incluyendo el presidente de Ecopetrol para la época de los hechos) y cuatro multinacionales contratistas del proyecto. De acuerdo con la decisión, en la modernización de la Refinería de Cartagena hubo mayores inversiones que no le agregaron valor al proyecto en cuantía de 997 millones de dólares (aproximadamente 2,9 billones de pesos), el monto en que cuantificó finalmente la Contraloría el daño fiscal ocasionado. (Le puede interesar: Contraloría ratifica cuestionamientos a cuentas de Reficar ) Además, el órgano de control encontró que hubo gastos que no tuvieron relación con el proceso de modernización de la Refinería, hubo una baja productividad de la mano de obra directa, mayores gastos en contratación de personal y demoras en las actividades propias del proyecto causadas por acciones y omisiones, por parte de la junta directiva de Reficar, la administración de Reficar, contratistas y supervisor del proyecto. Por ejemplo, se determinó que algunas de estas mayores inversiones consistieron en retrabajos, reprocesos, sobrantes excesivos en inventarios y gastos no asociados a la construcción. Para la entidad, el daño al patrimonio fue producto de las acciones y omisiones de la junta directiva de Reficar y su administración, contratista y supervisor del proyecto, quienes, dice la Contraloría , en ejercicio de la gestión fiscal directa o indirecta, de manera antieconómica, ineficiente e inoportuna, contribuyeron a la billonaria pérdida de recursos públicos. Sanciones por el control de cambio 2 y 3 Facebook Twitter Enviar Linkedin Para la Contraloría, en la modernización de la Refinería de Cartagena hubo mayores inversiones que no le agregaron valor al proyecto en cuantía de 997 millones de dólares. Foto: Yomaira Grandett / EL TIEMPO En primer lugar, por unas adiciones en exceso por 1,3 billones de pesos en el control de cambio 2, se falló responsabilidad fiscal, en forma solidaria y a título de culpa grave  en contra de los funcionarios de Reficar: Carlos Eduardo Bustillo Lacayo , vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar para la época de los hechos; Andrés Virgilio Riera Burelli, vicepresidente de proyectos de Reficar para la época; Reyes Reinoso Yanez , presidente y representante legal de Reficar para entonces; Orlando José Cabrales Martínez , también presidente y representante legal para ese momento; Magda Nancy Manosalva Cely , para entonces vicepresidenta financiera y administrativa. También fueron hallados responsables varios miembros de la junta directiva de la Refinería para ese momento: Javier Genaro Gutiérrez Pemberthy , presidente de Ecopetrol; Pedro Alfonso Rosales Navarro , en representación de Ecopetrol; Diana Constanza Calixto Hernández , en representación de Ecopetrol; Henry Medina González , miembro de la junta de Reficar y de Ecopetrol; y Hernando José Gómez Restrepo , miembro de la junta de Reficar. De otro lado, se halló fiscalmente responsables a los contratistas: Chicago Bridge & Iron Company CB&I UK Limited;  CBI Colombiana; Foster Wheeler USA Corporation y Process Consultants Inc. (Lea también: Expertos y analistas escriben sobre caso Reficar) De otro lado, por el daño patrimonial producido al aprobarse adiciones de recursos en exceso en cuantía de 1,6 billones de pesos en el control de cambio 3, se halló fiscalmente responsables, de forma solidaria y a título de culpa grave, a los funcionarios de Reficar para la época: Bustillo Lacayo, Riera Burelli, Reinoso Yanez, y Manosalva Cely. Así mismo, a los miembros de la junta directiva Gutiérrez Pemberthy, Rosales Navarro, Natalia Gutiérrez Jaramillo y Uriel Salazar Duque, ambos miembros de la junta directiva de Reficar. Los mismos cuatro contratistas fueron igualmente hallados responsables en este punto. Finalmente, se declaró como terceros civilmente responsables a las compañías de seguros Compañías Aseguradoras de Fianzas S.A. Confianza, Chubb de Colombia Compañía de Seguros S.A y AXA Colpatria Seguros S.A. Contra este fallo de responsabilidad fiscal, que es de primera instancia, se pueden interponer los recursos de reposición, ante la misma Contralora Delegada n.° 15 de la Unidad de Investigaciones Especiales Contra la Corrupción; y el de apelación ante la Sala Fiscal y Sancionatoria de la Contraloría General de la República. Cuando el fallo esté en firme, se enviará el expediente al Consejo de Estado para que ejerza el control automático e integral de legalidad previsto en el artículo 23 de la Ley 2080 de enero 25 de 2021. (Le recomendamos: Procuraduría archiva caso a exprocurador Carlos G. Arrieta por Reficar) Absueltos de responsabilidad fiscal Mientras en los controles de cambio 2 y 3 la Contraloría encontró responsabilidad fiscal, en el control de cambio 4, por 645.990 millones de pesos no fue así, por lo cual absolvió de responsabilidad fiscal a: Los miembros de la junta directiva de Reficar para la época, Astrid Martínez Ortiz, Carlos Gustavo Arrieta, Gutiérrez Pemberthy, Rosales Navarro, Salazar Duque, Reinoso Yanez, presidente de la junta. Así mismo, en favor de Riera Burelli, como vicepresidente del proyecto de expansión de Ecopetrol. También falló en favor, en este punto, de los contratistas CBI Colombiana, Chicago Bridge & Iron Company CB&I UK Limited; Foster Wheeler USA Corporation y Process Consultants Inc. Fueron absueltos de responsabilidad igualmente, por los controles de cambio 2, 3 y 4  Chicago Bridge & Iron Company (CB&I) Américas Ltd. y César Luis Barco García, director Corporativo de Proyectos de Ecopetrol. justicia@eltiempo.com En Twitter: @JusticiaET
    """}

chat_completion = llm_chain_NER(input, return_only_outputs=True)
print(chat_completion["text"])

{
    "Organizaciones": [
        "Contraloría General de la República",
        "Reficar",
        "Ecopetrol",
        "Chicago Bridge & Iron Company CB&I UK Limited",
        "CBI Colombiana",
        "Foster Wheeler USA Corporation",
        "Process Consultants Inc.",
        "Compañías Aseguradoras de Fianzas S.A. Confianza",
        "Chubb de Colombia Compañía de Seguros S.A",
        "AXA Colpatria Seguros S.A",
        "Consejo de Estado"
    ],
    "Individuos": [
        {
            "Nombre": "Carlos Eduardo Bustillo Lacayo",
            "Cargo": "Vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar"
        },
        {
            "Nombre": "Andrés Virgilio Riera Burelli",
            "Cargo": "Vicepresidente de proyectos de Reficar"
        },
        {
            "Nombre": "Reyes Reinoso Yanez",
            "Cargo": "Presidente y representante legal de Reficar"
        },
        {
            "Nombre": "Orlando José Cabrales Martínez",
   

In [40]:
import json
my_json = json.loads(chat_completion["text"])


In [67]:
my_json

{'Organizaciones': ['Contraloría General de la República',
  'Reficar',
  'Ecopetrol',
  'Chicago Bridge & Iron Company CB&I UK Limited',
  'CBI Colombiana',
  'Foster Wheeler USA Corporation',
  'Process Consultants Inc.',
  'Compañías Aseguradoras de Fianzas S.A. Confianza',
  'Chubb de Colombia Compañía de Seguros S.A',
  'AXA Colpatria Seguros S.A',
  'Consejo de Estado'],
 'Individuos': [{'Nombre': 'Carlos Eduardo Bustillo Lacayo',
   'Cargo': 'Vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar'},
  {'Nombre': 'Andrés Virgilio Riera Burelli',
   'Cargo': 'Vicepresidente de proyectos de Reficar'},
  {'Nombre': 'Reyes Reinoso Yanez',
   'Cargo': 'Presidente y representante legal de Reficar'},
  {'Nombre': 'Orlando José Cabrales Martínez',
   'Cargo': 'Presidente y representante legal de Reficar'},
  {'Nombre': 'Magda Nancy Manosalva Cely',
   'Cargo': 'Vicepresidenta financiera y administrativa de Reficar'},
  {'Nombre': 'Javier Genaro Gutiérrez Pembert

In [70]:
individuos = [item["Nombre"] for item in my_json['Individuos']]

In [73]:
individuos = ', '.join(individuos)

In [74]:
individuos

'Carlos Eduardo Bustillo Lacayo, Andrés Virgilio Riera Burelli, Reyes Reinoso Yanez, Orlando José Cabrales Martínez, Magda Nancy Manosalva Cely, Javier Genaro Gutiérrez Pemberthy, Pedro Alfonso Rosales Navarro, Diana Constanza Calixto Hernández, Henry Medina González, Hernando José Gómez Restrepo, Natalia Gutiérrez Jaramillo, Uriel Salazar Duque, Astrid Martínez Ortiz, Carlos Gustavo Arrieta'

# Extracción de Relaciones

In [88]:
TEMPLATE_ER = """
Como periodista y científico de redes, está analizando la red social que surgió de un Escándalo de corrupción en Colombia.
Su tarea es inferir y reconocer las relaciones entre las entidades nombradas SOLO de los siguientes individuos.
{individuos}

La salida solo deberia ser en formato JSON con las siguientes claves:

Nombre_Individuo_A: Nombre de los individuos que se relacionan
Detalles Relación: Detalles de la relación
Relación: Clasificador de la relación
Nombre_Individuo_B: Nombre de los individuos que se relacionan

Si no hay relación entre dos individuos, no escriba nada.
Los clasificadores de relaciones deben tener una frase verbal como ejemplo (nacio) + frase preposicional (EnCiudad) -> nacioenCiudad:\n 

La tarea se realiza en base al siguiente texto de una noticia:

```
{article}
```
"""

PROMPT_ER = PromptTemplate(input_variables=["individuos", "article"], template=TEMPLATE_ER)

In [91]:
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.2, max_tokens=2000)

llm_chain_ER = LLMChain(
    llm=llm,
    prompt=PROMPT_ER,
    #output_parse_function=parse_output,
)

In [92]:
llm

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', temperature=0.2, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=2000)

In [52]:
input.update({"individuos": individuos})

In [53]:
input

{'article': '\nLuego de una extensa investigación, la Contraloría General de la República profirió un fallo de responsabilidad fiscal por más de 2,9 billones de pesos por el daño producido a los intereses del Estado debido, según se lee en el documento, a las decisiones sobre el manejo de los recursos públicos que "conllevaron a la pérdida de valor de las mayores inversiones en el Proyecto de Ampliación y Modernización de la Refinería de Cartagena ( Reficar )". La decisión se conoció en un auto firmado este 26 de abril por la Contralora Delegada Intersectorial n.° 15 de la\xa0Unidad de Investigaciones Especiales contra la Corrupción. (Puede ser de su interés: Recursos por $ 6,6 billones están en juego en obras inconclusas) Es así como, por el mayor daño patrimonial al Estado en su historia, el órgano de control llamó a\xa0responder fiscalmente a dos\xa0presidentes y tres vicepresidentes de Reficar, siete\xa0miembros de su junta directiva (incluyendo el presidente de Ecopetrol para la é

In [93]:
chat_completion_ER = llm_chain_ER(input, return_only_outputs=True)
print(chat_completion_ER["text"])

{
  "Relaciones": [
    {
      "Nombre_Individuo_A": "Carlos Eduardo Bustillo Lacayo",
      "Detalles_Relación": "Vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar",
      "Relación": "asesoradoPor",
      "Nombre_Individuo_B": "Andrés Virgilio Riera Burelli"
    },
    {
      "Nombre_Individuo_A": "Carlos Eduardo Bustillo Lacayo",
      "Detalles_Relación": "Vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar",
      "Relación": "asesoradoPor",
      "Nombre_Individuo_B": "Reyes Reinoso Yanez"
    },
    {
      "Nombre_Individuo_A": "Carlos Eduardo Bustillo Lacayo",
      "Detalles_Relación": "Vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar",
      "Relación": "asesoradoPor",
      "Nombre_Individuo_B": "Orlando José Cabrales Martínez"
    },
    {
      "Nombre_Individuo_A": "Carlos Eduardo Bustillo Lacayo",
      "Detalles_Relación": "Vicepresidente de Proyecto y Asesor de la Vicepreside

In [95]:
my_json = json.loads(chat_completion_ER["text"])


In [97]:
pd.DataFrame(my_json["Relaciones"])

,Nombre_Individuo_A,Detalles_Relación,Relación,Nombre_Individuo_B
0,Carlos Eduardo Bustillo Lacayo,Vicepresidente de Proyecto y Asesor de la Vice...,asesoradoPor,Andrés Virgilio Riera Burelli
1,Carlos Eduardo Bustillo Lacayo,Vicepresidente de Proyecto y Asesor de la Vice...,asesoradoPor,Reyes Reinoso Yanez
2,Carlos Eduardo Bustillo Lacayo,Vicepresidente de Proyecto y Asesor de la Vice...,asesoradoPor,Orlando José Cabrales Martínez
3,Carlos Eduardo Bustillo Lacayo,Vicepresidente de Proyecto y Asesor de la Vice...,asesoradoPor,Magda Nancy Manosalva Cely
4,Andrés Virgilio Riera Burelli,Vicepresidente de proyectos de Reficar,trabajóCon,Reyes Reinoso Yanez
5,Andrés Virgilio Riera Burelli,Vicepresidente de proyectos de Reficar,trabajóCon,Orlando José Cabrales Martínez
6,Andrés Virgilio Riera Burelli,Vicepresidente de proyectos de Reficar,trabajóCon,Magda Nancy Manosalva Cely
7,Reyes Reinoso Yanez,Presidente y representante legal de Reficar,trabajóCon,Orlando José Cabrales Martínez
8,Reyes Reinoso Yanez,Presidente y representante legal de Reficar,trabajóCon,Magda Nancy Manosalva Cely
9,Orlando José Cabrales Martínez,Presidente y representante legal de Reficar,trabajóCon,Magda Nancy Manosalva Cely


In [78]:
import pandas as pd

if chat_completion_ER["text"]:
    # split response into list of lines
    lines = chat_completion_ER["text"].split("\n")
    # remove empty lines
    lines = [line for line in lines if line]
    # remove first line
    lines = lines[1:]
    # split each line into list of elements
    lines = [line.split(";") for line in lines]
    # create dataframe
    df = pd.DataFrame(lines, columns=["Nombre_Individuo_A", "Relación", "Nombre_Individuo_B"])
    # save dataframe to csv
    df.to_csv("../data/extracted_relationships/" + "output" + ".csv", sep=";", index=False)

In [79]:
df

,Nombre_Individuo_A,Relación,Nombre_Individuo_B
0,Carlos Eduardo Bustillo Lacayo,ColaboraciónEnFraude,Reyes Reinoso Yanez
1,Carlos Eduardo Bustillo Lacayo,ColaboraciónEnFraude,Orlando José Cabrales Martínez
2,Carlos Eduardo Bustillo Lacayo,ColaboraciónEnFraude,Magda Nancy Manosalva Cely
3,Andrés Virgilio Riera Burelli,ColaboraciónEnFraude,Reyes Reinoso Yanez
4,Andrés Virgilio Riera Burelli,ColaboraciónEnFraude,Orlando José Cabrales Martínez
5,Andrés Virgilio Riera Burelli,ColaboraciónEnFraude,Magda Nancy Manosalva Cely
6,Reyes Reinoso Yanez,ColaboraciónEnFraude,Orlando José Cabrales Martínez
7,Reyes Reinoso Yanez,ColaboraciónEnFraude,Magda Nancy Manosalva Cely
8,Orlando José Cabrales Martínez,ColaboraciónEnFraude,Magda Nancy Manosalva Cely
9,Javier Genaro Gutiérrez Pemberthy,MiembroDeJuntaDirectiva,Pedro Alfonso Rosales Navarro


# Use Pydantic

In [1]:
# LLMChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.chains import SequentialChain

# output parser with pydantic
from langchain.output_parsers import PydanticOutputParser
from typing import List
from pydantic import BaseModel, Field


# Cadena de las entidades nombradas

## Pydantic Model
class Individuo(BaseModel):
    Nombre: str = Field(description="Nombre del individuo")
    Cargo: str = Field(description="Cargo del individuo")
    RolEnEscandalo: str = Field(description="Rol en el escandalo")

class Relacion(BaseModel):
    Individuo: str = Field(description="Individuo en el escandalo")
    Relación: str = Field(description="Clasificador de la relación")
    Organización: str = Field(description="Organización en el escandalo")

class DataModelNER(BaseModel):
    Organizaciones: List[str] = Field(description="Lista de Nombres de Organizaciones")
    Individuos: List[Individuo] = Field(description="Lista de diccionarios con los nombres de los individuos y su rol en el escándalo")
    Relaciones: List[Relacion] = Field(description="Lista de triplas de las relaciones entre individuos y organizaciones")

parserNER = PydanticOutputParser(pydantic_object=DataModelNER)

## Prompt
TEMPLATE_NER = """
Como periodista y científico de redes, está analizando la red social que surgió de un Escándalo de corrupción en Colombia.
Su tarea es inferir y reconocer las entidades nombradas de organizaciones e individuos de la noticia entre ```.
Algunas de las entidades no son explicitas, por lo que debe inferirlas a partir de la información de la noticia.

La salida solo deberia ser en formato JSON con las siguientes claves:
{format_instructions}
    
Los clasificadores de relaciones deben tener una frase verbal como ejemplo (nacio) + frase preposicional (EnCiudad) -> nacioenCiudad:\n 

```
{article}
```
"""

PROMPT_NER = PromptTemplate(
    input_variables=["article"],
    template=TEMPLATE_NER,
    partial_variables={"format_instructions": parserNER.get_format_instructions()},
    )

## Definición de la Cadena
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.2, max_tokens=8000)

llm_chain_NER = LLMChain(
    llm=llm,
    prompt=PROMPT_NER,
    #output_parse_function=parse_output,
)

In [2]:
## Ejemplo
input = {"article":"""
Luego de una extensa investigación, la Contraloría General de la República profirió un fallo de responsabilidad fiscal por más de 2,9 billones de pesos por el daño producido a los intereses del Estado debido, según se lee en el documento, a las decisiones sobre el manejo de los recursos públicos que "conllevaron a la pérdida de valor de las mayores inversiones en el Proyecto de Ampliación y Modernización de la Refinería de Cartagena ( Reficar )". La decisión se conoció en un auto firmado este 26 de abril por la Contralora Delegada Intersectorial n.° 15 de la Unidad de Investigaciones Especiales contra la Corrupción. (Puede ser de su interés: Recursos por $ 6,6 billones están en juego en obras inconclusas) Es así como, por el mayor daño patrimonial al Estado en su historia, el órgano de control llamó a responder fiscalmente a dos presidentes y tres vicepresidentes de Reficar, siete miembros de su junta directiva (incluyendo el presidente de Ecopetrol para la época de los hechos) y cuatro multinacionales contratistas del proyecto. De acuerdo con la decisión, en la modernización de la Refinería de Cartagena hubo mayores inversiones que no le agregaron valor al proyecto en cuantía de 997 millones de dólares (aproximadamente 2,9 billones de pesos), el monto en que cuantificó finalmente la Contraloría el daño fiscal ocasionado. (Le puede interesar: Contraloría ratifica cuestionamientos a cuentas de Reficar ) Además, el órgano de control encontró que hubo gastos que no tuvieron relación con el proceso de modernización de la Refinería, hubo una baja productividad de la mano de obra directa, mayores gastos en contratación de personal y demoras en las actividades propias del proyecto causadas por acciones y omisiones, por parte de la junta directiva de Reficar, la administración de Reficar, contratistas y supervisor del proyecto. Por ejemplo, se determinó que algunas de estas mayores inversiones consistieron en retrabajos, reprocesos, sobrantes excesivos en inventarios y gastos no asociados a la construcción. Para la entidad, el daño al patrimonio fue producto de las acciones y omisiones de la junta directiva de Reficar y su administración, contratista y supervisor del proyecto, quienes, dice la Contraloría , en ejercicio de la gestión fiscal directa o indirecta, de manera antieconómica, ineficiente e inoportuna, contribuyeron a la billonaria pérdida de recursos públicos. Sanciones por el control de cambio 2 y 3 Facebook Twitter Enviar Linkedin Para la Contraloría, en la modernización de la Refinería de Cartagena hubo mayores inversiones que no le agregaron valor al proyecto en cuantía de 997 millones de dólares. Foto: Yomaira Grandett / EL TIEMPO En primer lugar, por unas adiciones en exceso por 1,3 billones de pesos en el control de cambio 2, se falló responsabilidad fiscal, en forma solidaria y a título de culpa grave  en contra de los funcionarios de Reficar: Carlos Eduardo Bustillo Lacayo , vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar para la época de los hechos; Andrés Virgilio Riera Burelli, vicepresidente de proyectos de Reficar para la época; Reyes Reinoso Yanez , presidente y representante legal de Reficar para entonces; Orlando José Cabrales Martínez , también presidente y representante legal para ese momento; Magda Nancy Manosalva Cely , para entonces vicepresidenta financiera y administrativa. También fueron hallados responsables varios miembros de la junta directiva de la Refinería para ese momento: Javier Genaro Gutiérrez Pemberthy , presidente de Ecopetrol; Pedro Alfonso Rosales Navarro , en representación de Ecopetrol; Diana Constanza Calixto Hernández , en representación de Ecopetrol; Henry Medina González , miembro de la junta de Reficar y de Ecopetrol; y Hernando José Gómez Restrepo , miembro de la junta de Reficar. De otro lado, se halló fiscalmente responsables a los contratistas: Chicago Bridge & Iron Company CB&I UK Limited;  CBI Colombiana; Foster Wheeler USA Corporation y Process Consultants Inc. (Lea también: Expertos y analistas escriben sobre caso Reficar) De otro lado, por el daño patrimonial producido al aprobarse adiciones de recursos en exceso en cuantía de 1,6 billones de pesos en el control de cambio 3, se halló fiscalmente responsables, de forma solidaria y a título de culpa grave, a los funcionarios de Reficar para la época: Bustillo Lacayo, Riera Burelli, Reinoso Yanez, y Manosalva Cely. Así mismo, a los miembros de la junta directiva Gutiérrez Pemberthy, Rosales Navarro, Natalia Gutiérrez Jaramillo y Uriel Salazar Duque, ambos miembros de la junta directiva de Reficar. Los mismos cuatro contratistas fueron igualmente hallados responsables en este punto. Finalmente, se declaró como terceros civilmente responsables a las compañías de seguros Compañías Aseguradoras de Fianzas S.A. Confianza, Chubb de Colombia Compañía de Seguros S.A y AXA Colpatria Seguros S.A. Contra este fallo de responsabilidad fiscal, que es de primera instancia, se pueden interponer los recursos de reposición, ante la misma Contralora Delegada n.° 15 de la Unidad de Investigaciones Especiales Contra la Corrupción; y el de apelación ante la Sala Fiscal y Sancionatoria de la Contraloría General de la República. Cuando el fallo esté en firme, se enviará el expediente al Consejo de Estado para que ejerza el control automático e integral de legalidad previsto en el artículo 23 de la Ley 2080 de enero 25 de 2021. (Le recomendamos: Procuraduría archiva caso a exprocurador Carlos G. Arrieta por Reficar) Absueltos de responsabilidad fiscal Mientras en los controles de cambio 2 y 3 la Contraloría encontró responsabilidad fiscal, en el control de cambio 4, por 645.990 millones de pesos no fue así, por lo cual absolvió de responsabilidad fiscal a: Los miembros de la junta directiva de Reficar para la época, Astrid Martínez Ortiz, Carlos Gustavo Arrieta, Gutiérrez Pemberthy, Rosales Navarro, Salazar Duque, Reinoso Yanez, presidente de la junta. Así mismo, en favor de Riera Burelli, como vicepresidente del proyecto de expansión de Ecopetrol. También falló en favor, en este punto, de los contratistas CBI Colombiana, Chicago Bridge & Iron Company CB&I UK Limited; Foster Wheeler USA Corporation y Process Consultants Inc. Fueron absueltos de responsabilidad igualmente, por los controles de cambio 2, 3 y 4  Chicago Bridge & Iron Company (CB&I) Américas Ltd. y César Luis Barco García, director Corporativo de Proyectos de Ecopetrol. justicia@eltiempo.com En Twitter: @JusticiaET
    """}

chat_completion = llm_chain_NER(input, return_only_outputs=True)

In [8]:
import json
json.loads(chat_completion['text'])

{'Organizaciones': ['Contraloría General de la República',
  'Reficar',
  'Ecopetrol',
  'Chicago Bridge & Iron Company CB&I UK Limited',
  'CBI Colombiana',
  'Foster Wheeler USA Corporation',
  'Process Consultants Inc.',
  'Compañías Aseguradoras de Fianzas S.A. Confianza',
  'Chubb de Colombia Compañía de Seguros S.A',
  'AXA Colpatria Seguros S.A'],
 'Individuos': [{'Nombre': 'Carlos Eduardo Bustillo Lacayo',
   'Cargo': 'Vicepresidente de Proyecto y Asesor de la Vicepresidencia de Proyecto de Reficar',
   'RolEnEscandalo': 'Responsable de adiciones en exceso en el control de cambio 2'},
  {'Nombre': 'Andrés Virgilio Riera Burelli',
   'Cargo': 'Vicepresidente de proyectos de Reficar',
   'RolEnEscandalo': 'Responsable de adiciones en exceso en el control de cambio 2 y 3'},
  {'Nombre': 'Reyes Reinoso Yanez',
   'Cargo': 'Presidente y representante legal de Reficar',
   'RolEnEscandalo': 'Responsable de adiciones en exceso en el control de cambio 2 y 3'},
  {'Nombre': 'Orlando Jos